In [1]:
!pip install inaSpeechSegmenter 'https://github.com/Numenorean/ShazamAPI/archive/master.zip' loguru zhconv
!apt-get install aria2 ffmpeg
!wget https://github.com/nilaoda/BBDown/releases/download/1.5.4/BBDown_1.5.4_20221019_linux-x64.zip
!unzip BBDown_1.5.4_20221019_linux-x64.zip
!chmod +x BBDown
!./BBDown https://www.bilibili.com/video/BV1tY411r7GU/ --use-aria2c -F '<ownerMid>'
# colab 内存不够一次处理全部。
!ffmpeg -i '/content/10850238.mp4' -ss 00:00:00 -to 01:30:00 -c:v copy -c:a copy '/content/10850238.1.mp4'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ShazamAPI: filename=ShazamAPI-0.0.2-py3-none-any.whl size=9889 sha256=5f80f431fd83a8d072353c9f901c40649d9a52580be9a3cc4dad1ac7e989dec8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wu86gjhm/wheels/4c/86/8a/b365f1efb6f9bb273b13bc44c91fa6a6fb1139bfa1a5a6b815
  Created wheel for zhconv: filename=zhconv-1.4.3-py2.py3-none-any.whl size=208852 sha256=19ea0a

In [ ]:
!ffmpeg -i '/content/10850238.mp4' -ss 00:00:00 -to 01:30:00 -c:v copy -c:a copy '/content/10850238.1.mp4'

In [3]:
from os import makedirs, rename, listdir, system
from os.path import basename, splitext, dirname, exists, join, isfile

import ShazamAPI
from inaSpeechSegmenter import Segmenter
from loguru import logger
from zhconv import convert


@logger.catch
def segment(media: str, batch_size: int = 32, energy_ratio: float = 0.02):
    logger.info(f'开始为 {media} 分段。')
    return Segmenter(
        vad_engine='sm',  # 'smn': 'speech', 'music', 'noise' (better) ; 'sm': 'speech', 'music'
        detect_gender=False,  # 性别确认
        energy_ratio=energy_ratio,  # ?
        batch_size=batch_size  # 根据显卡情况决定
    )(media)


@logger.catch
def extract_music(
        segmentation,  # 切片信息
        music_segment_threshold: int = 60,  # 合并前音乐切片的最短时间
        segment_connect: int = 3,  # 两个分割小于这个时间则合并分割
        music_segment_threshold_final: int = 90,  # 合并后音乐切片的最短时间
):
    if segmentation is None:
        logger.warning('切片信息为空，可能在分割的时候发生了异常，请检查。')
        return
    # 将结果中被认为没有声音且在 2s 内的片段分割的切分合并。
    for i in range(len(segmentation) - 2, 0, -1):
        if segmentation[i][0] == 'noEnergy' and \
                segmentation[i][2] - segmentation[i][1] < 2 and \
                segmentation[i - 1][0] == segmentation[i + 1][0]:
            segmentation[i - 1] = (segmentation[i - 1][0], segmentation[i - 1][1], segmentation[i + 1][2])

    r = list(filter(lambda x: x[0] == 'music' and x[2] - x[1] > music_segment_threshold, segmentation))

    # 合并过短的分割
    for i in range(len(r) - 1, 0, -1):
        if r[i][1] - r[i - 1][2] < segment_connect:
            r[i - 1] = (r[i - 1][0], r[i - 1][1], r[i][2])
            del r[i]

    rf = list(filter(lambda x: x[2] - x[1] > music_segment_threshold_final, r))
    return [
        [
            f'{str(int(x[1] // 3600)).zfill(2)}:{str(int(x[1] % 3600 // 60)).zfill(2)}:{str(int(x[1] % 60)).zfill(2)}',
            f'{str(int(x[2] // 3600)).zfill(2)}:{str(int(x[2] % 3600 // 60)).zfill(2)}:{str(int(x[2] % 60)).zfill(2)}',
        ] for x in rf
    ]


@logger.catch
def extract_mah_stuff(
        media,
        segmented_stamps,
        result_ext=None,  # 如果不提供则保留原始格式
        output_dir=None  # 如果不提供则使用原始文件所在目录下的 segmented
):
    if segmented_stamps is None:
        return

    if output_dir is None:
        output_dir = join(dirname(media), 'segmented')
    if not exists(output_dir):
        makedirs(output_dir)
    logger.info(f'将分割结果写入到 {output_dir}。')
    filename = basename(media)
    filename_without_ext, file_ext = splitext(filename)
    if result_ext is None:
        result_ext = file_ext

    logger.info(f'共 {len(segmented_stamps)} 段内容被识别。')
    for i in range(len(segmented_stamps)):
        system(
            f'ffmpeg -i "{media}" '
            f'-ss {segmented_stamps[i][0]} '
            f'-to {segmented_stamps[i][1]} '
            f'-c:v copy '
            f'-c:a copy '
            f'"{join(output_dir, filename_without_ext)}_{i}{result_ext}"')
    return output_dir


@logger.catch
def shazam(mp3, stop_at_first_match=True):
    logger.info(f'开始识别 {mp3} 。')
    recognize_generator = ShazamAPI.Shazam(
        open(mp3, 'rb').read(),
        lang='cn',
        time_zone='Asia/Shanghai'
    ).recognizeSong()

    matches = []
    try:
        while True:
            match = next(recognize_generator)
            if match[1].get('matches') and len(match[1].get('matches')) > 0 and match[1].get('track'):
                matches.append(match)
                if stop_at_first_match: raise StopIteration()
    except StopIteration:
        pass
    return matches


@logger.catch
def legalize_filename(file_name):
    return file_name.replace(':', ' ').replace('"', '').replace(r'/', '').replace(r'?', '').replace(r'*', '')


@logger.catch
def shazam_title(match):
    return legalize_filename(match[1]['track']['title']) + '_' + legalize_filename(match[1]['track']['subtitle'])


@logger.catch
def recognize_song(song_dir: str, dist_dir: str):
    if song_dir is None:
        return
    if not exists(dist_dir):
        makedirs(dist_dir)
    logger.info(f'移动结果到 {dist_dir}。')
    for file in listdir(song_dir):
        file_path = join(song_dir, file)
        if not isfile(file_path):
            continue
        recognize_result = shazam(file_path)
        if recognize_result:
            title = convert(shazam_title(recognize_result[0]), 'zh-cn')
            logger.info(f'识别结果为: {title} 。')
            filename = basename(file_path)
            filename_without_ext, file_ext = splitext(filename)
            rename(file_path, join(dist_dir, filename_without_ext) + '_' + title + file_ext)
        else:
            logger.warning(f'识别失败。')

In [4]:
raw_file_path = '/content/10850238.1.mp4'
seg_out_dir = r'/content/convert2music'
recognized_dir = r'/content/recognized'

extracted_info = extract_music(segment(raw_file_path, batch_size=512))
logger.info(f'分段情况：{extracted_info}。')
this_seg_out_dir = extract_mah_stuff(raw_file_path, extracted_info, output_dir=seg_out_dir)
recognize_song(this_seg_out_dir, recognized_dir)

2023-02-28 12:28:10.946 | INFO     | __main__:segment:12 - 开始为 /content/10850238.1.mp4 分段。


5885888/5885888 [==============================] - 0s 0us/step


/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/inaSpeechSegmenter/segmenter.py:62: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asany

499/499 - 15s - 15s/epoch - 31ms/step


2023-02-28 12:29:06.128 | INFO     | __main__:<module>:6 - 分段情况：[['00:03:36', '00:07:47'], ['00:07:54', '00:11:47'], ['00:14:14', '00:18:48'], ['00:20:57', '00:22:48'], ['00:26:21', '00:28:44'], ['00:29:14', '00:30:58'], ['00:32:01', '00:34:50'], ['00:40:13', '00:45:05'], ['00:55:41', '00:59:26'], ['01:04:58', '01:06:42'], ['01:10:08', '01:13:04'], ['01:14:59', '01:18:07']]。
2023-02-28 12:29:06.130 | INFO     | __main__:extract_mah_stuff:69 - 将分割结果写入到 /content/convert2music。
2023-02-28 12:29:06.134 | INFO     | __main__:extract_mah_stuff:75 - 共 12 段内容被识别。
2023-02-28 12:29:14.166 | INFO     | __main__:recognize_song:124 - 移动结果到 /content/recognized。
2023-02-28 12:29:14.169 | INFO     | __main__:shazam:89 - 开始识别 /content/convert2music/10850238.1_3.mp4 。
2023-02-28 12:29:28.676 | INFO     | __main__:recognize_song:132 - 识别结果为: 我不难过_Yanzi Sun 。
2023-02-28 12:29:28.678 | INFO     | __main__:shazam:89 - 开始识别 /content/convert2music/10850238.1_7.mp4 。
2023-02-28 12:31:11.717 | WARNING  | __main

In [5]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/594461
!mv /content/recognized /content/drive/MyDrive/594461
!mv /content/convert2music /content/drive/MyDrive/594461

Mounted at /content/drive
